In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
import datetime
import regex


In [ ]:
additionalstopwords = ["Deutsche","Deutsch","Deutschen","Deutscher"]
stop_words = stopwords.words("german") +additionalstopwords



In [ ]:
import glob
datafilepaths = glob.glob("MorerecentData/*")

In [ ]:
customerevaluations = pd.DataFrame(columns=['company','date','headline','body','rating'])

for files in datafilepaths:
    customerevaluation = pd.read_csv(files)
    customerevaluations = pd.concat([customerevaluations,customerevaluation],ignore_index=True)

customerevaluations["date"] = pd.to_datetime(customerevaluations["date"])
customerevaluations.drop_duplicates(["headline","body","rating"])
customerevaluations["length of review"] = customerevaluations["body"].str.len()
customerevaluations['body'] = customerevaluations['body'].map(lambda x: x.lower())
customerevaluations["rating"] = pd.to_numeric(customerevaluations['rating'])
customerevaluations["originalbody"] = customerevaluations['body']
customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub('&nt', '', x))
banknames = ["C24 Bank","deutsche ing (diba)","bank c24","C24","Deutsche Bank","ING Deutschland","bank ing-diba","ing diba bank","ing-diba bank","bank ing","ingdiba","ing-diba","ing diba","ing bank","ing  diba","n26 bank","bank n26"
,"N26","Sparda-Bank BW","Sparda-Bank Südwest eG","Volksbank","Commerzbank","dkb bank","bank dkb","DKB","HypoVereinsbank","Hypo"
,"Postbank","PSD Braunschweig","PDF Rhein Ruhr","PSD Karlsruhe","PSD berlin brandenburg","Sparda-Bank Hamburg","Sparda-Bank München","sparda- banken","Sparda-Bank Nürnberg"
,"Sparkasse Hannover","Volksbank Mittelhessen","volksbank Berlin","Volksbank Hamburg","Sparkasse Köln bonn",
"sparkasse bank","Sparkasse","sparkasse","Sparda-Bank",
"spardap","sparda bank","sparda banken","psd bank","psd","targobank","revolut","targo bank","tomorrow","vivid money","vividmoney","vivid-money","vivid","fidor bank","fidor","comdirect","norisbank","santander","noris bank","bunq","Fidorbank","fidor bank"
" diba ","dkb","Diba","bank deutschen bank","deutsche Bank","deutschen bank"," ING ","Bankbank",
"bank bank",]
for word in banknames:
    customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub(word.lower(), 'Bank', x))

for word in stop_words:
    customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub(" "+word.lower()+" ", ' ', x))

countrynames = ["Deutschland","Österreich"]
citynames = ["Berlin","Frankfurt"]

for word in countrynames:
    customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub(word.lower(), 'Land', x))

for word in citynames:
    customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub(word.lower(), 'Stadt', x))

customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub('&', '', x))
customerevaluations['word count'] = customerevaluations['body'].map(lambda x: len(x.strip().split(" ")))

customerevaluations.groupby(["company"]).describe()





In [ ]:
#mois kommenatre ausfiltern
searchfor = ['mois',"Mois","MOIS","keller","Keller","nahuiiiiiiiiiiii","lakakakakslskdnwkskd","einfach aus prinzip nahuyyyyyyy",
"nahui","oh yeah nahui","nahuiiiii","kellwerwärter","schnelllgmarceletkevin","83 jahre alt und nach schlaganfall pflegebedürftig","mühlhshabbshsbshyjabvsuabsbsjabasjss","de54100110012621571864","huens","ich kunde der deutschen bank (odenkirchener str. 43, 41236 mönchengladbach)","bernlöhr66538","inshallah","ist ja ekelhaft","drecksverein.geht den bach runter.","leute geht niemals zur Bank der gröste müll","gröster müll der welt.",".yyyyyyyyyyyyyyyyyy","oaschlöcher","ja sie wollen, dass ich falle lele"
,"gelber oasch nahoi","ya kelb ihr seid einfach lächerlich","jetzt werden önkel gefickt najui","alles neider","richtige lipp lipps die gehören gehauen","wiiixxxxxxxerrrr","erika","wenn koi luscht hen koi zeit hen wedder bleed ish en shineesischer saddelit uf erd stertze dud, konschde deene wadde, un wadde, un wadde, un..., bassiert nix. hen ja fielleischt nerffe","xxxxxxxxxcccccxcxc","einfach aus prinzip nahuyyyyyyy"]
customerevaluations = customerevaluations[~customerevaluations.body.str.contains('|'.join(searchfor))]
customerevaluations['date'] = customerevaluations['date'].apply(lambda a: pd.to_datetime(a).date()) 
customerevaluations = customerevaluations[(customerevaluations["date"]< (datetime.date(2022,4,16)))]
# still mention in research marketing start?

In [ ]:
customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub('\d{1,},\d{1,}.€', 'Betrag', x))
customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub('\d{1,}.euro', 'Betrag', x))

In [ ]:
customerevaluations.describe()

In [ ]:
customerevaluations.groupby("rating").mean()

In [ ]:
# customerevaluations = customerevaluations[customerevaluations["company"]== "Volksbank"]

In [ ]:
from bertopic import BERTopic


In [ ]:
customerevaluations.reset_index(drop=True, inplace=True)
topic_model = BERTopic(language="german",calculate_probabilities=False,top_n_words=10,nr_topics="auto")
topics, probs = topic_model.fit_transform(customerevaluations["body"])


In [ ]:
#topics_over_time = topic_model.topics_over_time(customerevaluations["body"], topics, customerevaluations["date"], nr_bins=20)
#topics_per_class = topic_model.topics_per_class(customerevaluations["body"],topics,classes= customerevaluations["company"])

In [ ]:

topic_freq = topic_model.get_topic_freq()
outliers = topic_freq['Count'][topic_freq['Topic']==-1].iloc[0]
print(f"{outliers} documents have not been classified")
print(f"The other {topic_freq['Count'].sum() - outliers} documents are {topic_freq['Topic'].shape[0]-1} topics")

In [ ]:
customerevaluations["topic"] = topics

In [ ]:
topic_model.get_topic(0)

In [ ]:
t = topic_model.get_topic(1)
len(t)

In [ ]:

topicsAndTopWords = []
#for i in range(1,len(topic_freq["Topic"])):
for i in range(0,len(topic_freq["Topic"])-1):
    t = topic_model.get_topic(i)
    words = ""

    for w in range(0,len(t)-1):
        words = words + "_" + t[w][0] 
    topicsAndTopWords.append(words)

topicsAndTopWords = pd.DataFrame({"top words": topicsAndTopWords})



In [ ]:
topic_model.visualize_barchart(n_words=10,top_n_topics=60)

In [ ]:
#topic_model.update_topics(customerevaluations["body"], topics, n_gram_range=(1, 1))

In [ ]:
new_topics, new_probs= topic_model.reduce_topics(customerevaluations["body"], topics, nr_topics=50)


In [ ]:
topic_model.visualize_barchart(n_words=10,top_n_topics=60)

In [ ]:

customerevaluations["reducedtopic"] = new_topics
reducedtopic_freq = topic_model.get_topic_freq()

reducedtopicsAndTopWords = []
#for i in range(1,len(topic_freq["Topic"])):
for i in range(0,len(reducedtopic_freq["Topic"])-1):
    t = topic_model.get_topic(i)
    words = ""

    for w in range(0,len(t)-1):
        words = words + "_" + t[w][0] 
    reducedtopicsAndTopWords.append(words)

reducedtopicsAndTopWords = pd.DataFrame({"top words": reducedtopicsAndTopWords})


In [ ]:

with pd.ExcelWriter('Model_09_06_2022_size20_nrtopicsauto_reducedto50.xlsx') as writer:  
    customerevaluations.to_excel(writer, sheet_name='evaluations',engine="xlsxwriter")
    topicsAndTopWords.to_excel(writer, sheet_name='topics',engine="xlsxwriter")
    reducedtopicsAndTopWords.to_excel(writer, sheet_name='reduced_topics')

In [ ]:

'''#len(reduction[0]) #to reduce topics then before doing everything with reduced model.
#topic_model.save("Mymodel15topnword")
with pd.ExcelWriter('sentimentanalysis.xlsx') as writer:  
    diccdf.to_excel(writer, sheet_name='Wordlist')
    
'''

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
topic_model.visualize_heatmap()

In [ ]:
topic_model.save("Model_09_06_2022_size20_nrtopicsauto_reducedto50")

In [ ]:
topic_modeltestreload = BERTopic.load("my_model")

In [ ]:
topic_modeltestreload.get_topic(1)

In [ ]:
diccfilepaths = glob.glob("Dictionaries German language/*")


#sentdict = pd.read_csv(diccfilepaths[0])

diccdf1 = pd.read_csv(
diccfilepaths[0], sep="\t",header=None)
diccdf2 = pd.read_csv(
diccfilepaths[1], sep="\t",header=None)


diccdf = pd.concat([diccdf2,diccdf1],ignore_index=True)
    

diccdf[0] = diccdf[0].map(lambda x: x[0:x.find("|")]+",")
diccdf["words"] = (diccdf[0]+diccdf[2]).astype(str)
diccdf.fillna("nan",inplace=True)

In [ ]:
wordlist =[]
countsofwords = []
iteration = -1
for row in diccdf["words"]:
    iteration = iteration +1
    if row !="nan":
        print(row)
        wordlist = str(row).split(",")
        shortcount = []
        
        for word in wordlist:
            
            shortcount.append(customerevaluations['body'].str.count(" "+word.lower()+" ").sum())
        countsofwords.append(sum(shortcount))
    if row =="nan":
        singleword = (diccdf[0][iteration])[:-1]
        print("NAN value    " +  singleword)
        countsofwords.append(customerevaluations['body'].str.count(str(" "+singleword.lower()+" ")).sum())


#for word in diccdf[0]:
    #countsofwords.append(customerevaluations['body'].str.count(word).sum())

diccdf["counts"] = countsofwords
